# Analiza wskaźników na daily

In [1]:
from tvDatafeed import TvDatafeed, Interval
import pandas as pd
from scipy.stats import chi2_contingency
import pandas_ta as ta

In [2]:
tv = TvDatafeed()
d = tv.get_hist("DAX", "XETR", Interval.in_daily, n_bars=5000).drop(["symbol", "volume"], axis=1).reset_index()

you are using nologin method, data you access may be limited


In [3]:
d["date"] = pd.to_datetime(d.datetime).dt.date
d = d.drop("datetime", axis=1)

In [4]:
d["daydir"] = d.apply(lambda r: "up" if r.close >= r.open else "down", axis=1)

In [5]:
def getpos(d, x, bench):
    name = x + "__vs__" + bench
    series = d.apply(
        lambda r: "up" if r[x] >= r[bench] else "down",
        axis=1
    )
    series.index = d.index
    series.name = name
    return series

def _bpos(x, u, l):
    if x >= u:
        return "up"
    if x <= l:
        return "down"
    return "in"
def getbandpos(d, x, up, low):
    name = x + "__vs__" + up + "_" +low
    series = d.apply(
        lambda r: _bpos(r[x], r[up], r[low]),
        axis=1
    )
    series.index = d.index
    series.name = name
    

In [6]:
smas = []
sma_names = []
for x in range(2, 31):
    series = ta.sma(d.close, x)
    series.index = d.index
    series.name = "sma" + str(x)
    smas.append(series)
    sma_names.append(series.name)
smas = pd.concat(smas, axis=1)
dsmas = pd.concat([d, smas], axis=1)

In [7]:
dsmas

,open,high,low,close,date,daydir,sma2,sma3,sma4,sma5,...,sma21,sma22,sma23,sma24,sma25,sma26,sma27,sma28,sma29,sma30
0,3533.3000,3536.0500,3327.9600,3332.6499,2002-08-05,down,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3319.7700,3601.5701,3235.3799,3568.6399,2002-08-06,up,3450.64490,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3570.1101,3632.4700,3440.5701,3465.5400,2002-08-07,down,3517.08995,3455.609933,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3473.4800,3679.2600,3473.4800,3679.2600,2002-08-08,up,3572.40000,3571.146633,3511.52245,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3682.7400,3760.8601,3607.5300,3760.8601,2002-08-09,up,3720.06005,3635.220033,3618.57500,3561.38998,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,14402.1600,14437.9100,14027.1800,14151.6900,2022-04-06,down,14288.02500,14364.736667,14385.17250,14391.08800,...,14260.427143,14195.476364,14136.310000,14092.902917,14077.1228,14074.160769,14067.890000,14081.930357,14098.664828,14097.112667
4996,14187.1300,14315.3500,14069.5300,14078.1500,2022-04-07,down,14114.92000,14218.066667,14293.09000,14323.76800,...,14271.390000,14252.141818,14190.375217,14133.886667,14092.3128,14077.162308,14074.308519,14068.256429,14081.800000,14097.981000
4997,14275.8700,14321.8200,14178.2800,14283.6700,2022-04-08,up,14180.91000,14171.170000,14234.46750,14291.20600,...,14311.464762,14271.948182,14253.512609,14194.262500,14139.8780,14099.672692,14084.810741,14081.785714,14075.684483,14088.529000
4998,14162.9700,14298.0200,14094.8300,14192.7800,2022-04-11,up,14238.22500,14184.866667,14176.57250,14226.13000,...,14338.353810,14306.070000,14268.506087,14250.982083,14194.2032,14141.912692,14103.121111,14088.666786,14085.613103,14079.587667


In [8]:
def getlags(d, cols, n):
    lags = []
    names = []
    for c in cols:
        for x in range(1, n+1):
            lag_name = c + "_lag_" + str(x)
            lag = d[c].shift(x)
            lag.name = lag_name
            lags.append(lag)
            names.append(lag_name)
    return pd.concat(lags, axis=1), names

In [17]:
lags, names = getlags(dsmas, sma_names, 30)
dsmalags = pd.concat((dsmas, lags), axis=1).dropna()
len(names)

870

In [19]:
def getpos(d, prices, bench):
    dirs = []
    names = []
    for p in prices:
        for b in bench:
            series = d.apply(
                lambda r: "up" if r[p] >= r[b] else "down",
                axis=1
            )
            series.name = p + "__vs__" + b
            names.append(series.name)
            dirs.append(series)
    return pd.concat(dirs, axis=1), names

In [20]:
dirs, dirnames = getpos(dsmalags, ["open"], names)

In [25]:
dsmadirs = pd.concat([dsmalags, dirs], axis=1).dropna()

In [28]:
dirnames[0]

'open__vs__sma2_lag_1'

In [33]:
ctab = pd.crosstab(dsmadirs['open__vs__sma2_lag_1'], dsmadirs.daydir)
print(ctab)
print(chi2_contingency(ctab))

daydir                down    up
open__vs__sma2_lag_1            
down                  1058  1102
up                    1277  1504
(4.453361678890616, 0.03483261356768514, 1, array([[1020.76502732, 1139.23497268],
       [1314.23497268, 1466.76502732]]))


In [75]:
conts = []
for d in dirnames:
    ctab = pd.crosstab(dsmadirs[d], dsmadirs.daydir)
    cont = chi2_contingency(ctab)
    if cont[1] <= 0.01:
       conts.append([d, cont])

In [76]:
sc = sorted(conts, key = lambda x: x[1][1])

In [82]:
best = sc[0][0]
indep  = []
for c in sc[1:]:
    ctab = pd.crosstab(dsmadirs[best], dsmadirs[c[0]])
    cont = chi2_contingency(ctab)
    indep.append([c, cont])

In [84]:
# They're all the same!

In [85]:
laggers = [s[0] for s in sc]

In [103]:
best_sum = 66666
for l in laggers:
    inter, lag = re.search(r"open__vs__sma(\d+)_lag_(\d+)", laggers[0]).groups()
    s = int(inter) + int(lag)
    if s <= best_sum:
        best_sum = s
        best_l = l

In [112]:
ctab = pd.crosstab(dsmadirs["open__vs__sma3_lag_1"], dsmadirs.daydir)
chi2_contingency(ctab)

(6.850366028471563,
 0.008862311431125673,
 1,
 array([[1008.00546448, 1124.99453552],
        [1326.99453552, 1481.00546448]]))